<center>
<p><img src="https://www.gob.mx/cms/uploads/image/file/179499/outstanding_quienes-somos.jpg" width="300">
</p>



# Curso *Machine Learning con uso de pandas, scikit learn y libretas jupyter*

# Probando diferentes métodos de regresión


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="80">
</p>
</center>




## Creando los conjuntos de datos a utilizar

Para este problema vamos a generar dos conjuntos de aprendizaje sintéticos que nos ayudarán a visualizar diferencias entre métodos. 

Es importante siempre tener en cuenta que, por cuestiones de visualización, sólo estamos poniendo una regresión en una sola dimensión.

Como hay métodos que son más efectivos en varias dimensiones, vamos a probar utilizando un conjunto multidimensional de datos reales, entre los conjuntos de juguete que vienen en *scikit-learn*.

In [ ]:
import numpy as np
import matplotlib.pyplot as pl


# Primer problema de regresión sintético
x1_plot = np.linspace(0, 10, 100)
y1_plot = x1_plot * np.sin(x1_plot)

x1 = np.linspace(0, 10, 100)
rng = np.random.RandomState(0)
rng.shuffle(x1)
x1 = np.sort(x1[:20])

y1 = x1 * np.sin(x1)
x1 = x1.reshape(-1, 1)

pl.figure(figsize=(15, 7))
pl.plot(x1_plot, y1_plot, label="Verdad")
pl.plot(x1, y1, 'o', label='Entrenamiento')
pl.title("Primer conjunto de prueba")
pl.legend()
pl.show()

In [ ]:
# Segundo problema de regresión sintético
x2_plot = np.linspace(0, 6, 100)
y2_plot = np.sin(x2_plot) + np.sin(6 * x2_plot)

rng = np.random.RandomState(1)
x2 = np.linspace(0, 6, 100)

y2 = np.sin(x2) + np.sin(6 * x2) + rng.normal(0, 0.1, x2.shape[0])
x2 = x2.reshape(-1, 1)

pl.figure(figsize=(15, 7))
pl.plot(x2_plot, y2_plot, label="Verdad")
pl.plot(x2, y2, 'o', label='Entrenamiento')
pl.title("Segundo conjunto de prueba")
pl.legend()
pl.show()

In [ ]:
# Tercer problema, con datos reales en 10 dimensiones

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

x, y = load_diabetes(return_X_y=True)
x3, x3_plot, y3, y3_plot = train_test_split(x, y, test_size=20, random_state=3)

print(f"El conjunto de entrenamiento tiene una forma {x3.shape}")
print(f"El conjunto de validación tiene una forma {x3_plot.shape}")

pl.figure(figsize=(15, 7))
pl.plot(y3_plot, 'ob', label="Verdad")
pl.title("Salidas del conjunto de prueba")
pl.show()





## Seleccionando los métodos de regresión que vamos a probar

Para este problema vamos a revisar diferentes métodos, los cuales vamos a iniciar con los parámetros ajustados con los valores básicos de la literatura, y poco a poco podremos irlos personalizando.

Estos son los métodos que vamos a utilizar:

**Aqui vamos a modificar si queremos probar que pasa modificando parámetros a los métodos o cambiando de método**

In [50]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


nombres = ['Rigida', 'Polinomial', 'SVR Lineal', 'SVR Gauss', 'SVR Poli', 'KNN', 
           'Arbol desición', 'Bósque aleatorio', 'GTB']
regresores = [
  Ridge(),
  make_pipeline(PolynomialFeatures(3), Ridge()),
  SVR(kernel='linear'),
  SVR(kernel='rbf'),
  SVR(kernel='poly'),
  KNeighborsRegressor(),
  DecisionTreeRegressor(),
  RandomForestRegressor(),
  GradientBoostingRegressor()
]


## Probando con los métodos seleccionados para regresión

Ahora vamos a aplicar los mñetodos a los datos y vamos a medir la bondad de los metodos usando la medida MAE (Mean Absolute Error). 

Veamos que pasa con los diferentes métodos:

In [ ]:
from sklearn.metrics import mean_absolute_error


pl.figure(figsize=(15,45))

pl.subplot(len(regresores) + 1, 3, 1)
pl.plot(x1_plot, y1_plot, label="Verdad")
pl.plot(x1, y1, 'o', label='Entrenamiento')
pl.title(r"$x \sin(x)$")

pl.subplot(len(regresores) + 1, 3, 2)
pl.plot(x2_plot, y2_plot, label="Verdad")
pl.plot(x2, y2, 'o', label='Entrenamiento')
pl.title(r"$\sin(x) + 6 \sin(x)$")

pl.subplot(len(regresores) + 1, 3, 3)
pl.plot(y3_plot, 'o', label='Datos de prueba')
pl.title("Problema de diabetes (10 variables)")

for i, regr in enumerate(regresores):
  regr.fit(x1, y1)
  y1_est = regr.predict(x1_plot.reshape(-1,1))
  mae = mean_absolute_error(y1_plot, y1_est)
  pl.subplot(len(regresores) + 1, 3, 3 * (i + 1) + 1)
  pl.plot(x1_plot, y1_est)
  pl.plot(x1, y1, 'o')
  pl.title(f"{nombres[i]} (MAE: {mae:.2f})")

  regr.fit(x2, y2)
  y2_est = regr.predict(x2_plot.reshape(-1,1))
  mae = mean_absolute_error(y2_plot, y2_est)
  pl.subplot(len(regresores) + 1, 3, 3 * (i + 1) + 2)
  pl.plot(x2_plot, y2_est)
  pl.plot(x2, y2, 'o')
  pl.title(f"{nombres[i]} (MAE: {mae:.2f})")

  regr.fit(x3, y3)
  y3_est = regr.predict(x3_plot)
  mae = mean_absolute_error(y3_plot, y3_est)
  pl.subplot(len(regresores) + 1, 3, 3 * (i + 1) + 3)
  pl.plot(y3_plot, 'o')
  pl.plot(y3_est, 'x')
  pl.title(f"{nombres[i]} (MAE: {mae:.2f})")
  




## Vamos ahora a practicar

1. Revisa en la documentación de *Scikit learn* las especificaciones y los parámetros que se pueden ajustar por cada uno de estos regresores. 

2. Prueba ajustar a cada uno de los problemas ¿Que encontraste? ¿Cuales son tus allazgos principales? ¿Que pasa cuando ajustas parámetros a un problema, con los otros problemas?

3. Registra el mejor regresor, con los mejores parámetros para cada problema.